In [8]:
import pickle
from typing import List

import numpy as np
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from importlib import reload
import visualise
reload(visualise)
from visualise import get_tsne_points, do_inference, PKL_PATH
from constants import *

init_notebook_mode(connected=True)

In [9]:
import pandas as pd


def clean_nans(embeddings_obj):
    clean_records = []
    clean_labels = []
    records = embeddings_obj["embeddings"]
    print(f"Got {records.shape[0]} records before cleaning")
    for i, record in enumerate(records):
        if not np.isnan(record).any():
            clean_records.append(record)
            clean_labels.append(embeddings_obj["labels"][i])
    records = np.stack(clean_records, axis=0)
    print(f"Got {records.shape[0]} records after cleaning")
    return {
        "embeddings": records,
        "labels": clean_labels
    }

def points2dataframe(points, labels):
    columns = ["x", "y"]
    points_df = pd.DataFrame(points, columns=columns)
    labels = [", ".join(l) for l in labels]
    points_df["labels"] = labels
    return points_df

def plot_embeddings(embeddings_df):
    fig = px.scatter(embeddings_df, x="x", y="y", hover_data="labels")
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        template='seaborn'
        
    )
    fig.show()



In [10]:

with open(PKL_PATH, "rb") as fh:
    embeddings_obj = pickle.load(fh)

embeddings_cleaned = clean_nans(embeddings_obj)

Got 324 records before cleaning
Got 312 records after cleaning


In [11]:
from visualise import get_tsne_points

embeddings = embeddings_cleaned["embeddings"]
labels = embeddings_cleaned["labels"]
points2d = get_tsne_points(embeddings)
emb_df = points2dataframe(points2d, embeddings_cleaned["labels"])
plot_embeddings(emb_df)